In [16]:
import os
import math
import requests
import faiss
import numpy as np
import concurrent.futures
import re
import time

In [2]:
from datetime import date
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_ollama.llms import OllamaLLM
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.chains import RetrievalQA

In [3]:
print("Alle Bibliotheken importiert und bereit.")

Alle Bibliotheken importiert und bereit.


In [4]:
MODEL_NAME = "deepseek-r1:7b"
INDEX_PATH = "faiss_index.faiss"
API_URL = "https://data.bka.gv.at/ris/api/v2.6/Landesrecht"
print(f"Konfiguration gesetzt: MODEL_NAME = {MODEL_NAME}, INDEX_PATH = '{INDEX_PATH}', API_URL = {API_URL}")

Konfiguration gesetzt: MODEL_NAME = deepseek-r1:7b, INDEX_PATH = 'faiss_index.faiss', API_URL = https://data.bka.gv.at/ris/api/v2.6/Landesrecht


In [5]:
embeddings = OllamaEmbeddings(model=MODEL_NAME)
llm = OllamaLLM(model=MODEL_NAME)
session = requests.Session()
print("Embeddings- und LLM-Objekte erstellt. Aktuelle Session wurde aufgerufen.")

Embeddings- und LLM-Objekte erstellt. Aktuelle Session wurde aufgerufen.


In [6]:
def fetch_and_parse_ris(ref):
    """
    1) Metadaten holen
    2) HTML-URL abrufen und Seite laden
    3) Mit BeautifulSoup nach dem <a>-Tag für das PDF suchen
    4) Absolute PDF-URL bauen
    5) PDF via PyPDFLoader einlesen und Text zurückliefern
    """
    lrKons = (
        ref.get("Data", {})
            .get("Metadaten", {})
            .get("Landesrecht", {})
            .get("LrKons", {})
    )
    html_url = lrKons.get("GesamteRechtsvorschriftUrl")
    if not html_url:
        return []

    try:
        resp = session.get(html_url, timeout=60)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")

        a_pdf = soup.find(
            "a",
            href=re.compile(r"\.pdf$"),
            title=lambda t: t and "PDF-Dokument" in t
        )
        if not a_pdf or not a_pdf.has_attr("href"):
            print(f"PDF-Link nicht gefunden in {html_url}")
            return []

        pdf_path = a_pdf["href"]
        pdf_url = urljoin("https://www.ris.bka.gv.at", pdf_path)

        print(f"Lade PDF: {pdf_url}")
        loader = PyPDFLoader(pdf_url)
        docs = loader.load()
        print(f"{len(docs)} Seiten geladen von {pdf_url}")
        return [(pdf_url, doc.page_content) for doc in docs]

    except Exception as e:
        print(f"Fehler beim Verarbeiten von {html_url}: {e}")
        return []

In [7]:
heute  = date.today().isoformat()
params = {
    "Applikation": "LrKons",
    "Bundesland_SucheInSalzburg": "true",
    "Sortierung_SortDirection": "Descending",
    "DokumenteProSeite": "Ten",
    "Fassung_FassungVom": heute
}
resp = session.get(API_URL, params=params)
resp.raise_for_status()
data = resp.json()
hits = int(data["OgdSearchResult"]["OgdDocumentResults"]["Hits"]["#text"])
page_size = int(data["OgdSearchResult"]["OgdDocumentResults"]["Hits"]["@pageSize"])
total = math.ceil(hits / page_size)

In [8]:
all_refs = []
# for page in range(1, total+1):
for page in range(1, 2):
    params["Seitennummer"] = page
    resp = session.get(API_URL, params=params); resp.raise_for_status()
    docs = resp.json()["OgdSearchResult"]["OgdDocumentResults"]["OgdDocumentReference"]
    all_refs.extend(docs if isinstance(docs, list) else [docs])
print(f"Insgesamt {len(all_refs)} Dokumentreferenzen geladen.")

Insgesamt 10 Dokumentreferenzen geladen.


In [9]:
all_sections = []
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as pool:
    futures = [pool.submit(fetch_and_parse_ris, ref) for ref in all_refs]
    for fut in concurrent.futures.as_completed(futures):
        all_sections.extend(fut.result() or [])
print(f"Insgesamt {len(all_sections)} Textabschnitte aus PDFs extrahiert.")

Lade PDF: https://www.ris.bka.gv.at/GeltendeFassung/LrSbg/20001469/Zweitwohnung-Beschr%c3%a4nkungsgemeinden-Verordnung%202024%2c%20Fassung%20vom%2017.06.2025.pdf
Lade PDF: https://www.ris.bka.gv.at/GeltendeFassung/LrSbg/20001469/Zweitwohnung-Beschr%c3%a4nkungsgemeinden-Verordnung%202024%2c%20Fassung%20vom%2017.06.2025.pdf
Lade PDF: https://www.ris.bka.gv.at/GeltendeFassung/LrSbg/20001469/Zweitwohnung-Beschr%c3%a4nkungsgemeinden-Verordnung%202024%2c%20Fassung%20vom%2017.06.2025.pdf
Lade PDF: https://www.ris.bka.gv.at/GeltendeFassung/LrSbg/20001389/%20ZWAG%2c%20Fassung%20vom%2017.06.2025.pdf
Lade PDF: https://www.ris.bka.gv.at/GeltendeFassung/LrSbg/20001389/%20ZWAG%2c%20Fassung%20vom%2017.06.2025.pdf
Lade PDF: https://www.ris.bka.gv.at/GeltendeFassung/LrSbg/20001389/%20ZWAG%2c%20Fassung%20vom%2017.06.2025.pdf
Lade PDF: https://www.ris.bka.gv.at/GeltendeFassung/LrSbg/20001389/%20ZWAG%2c%20Fassung%20vom%2017.06.2025.pdf
Lade PDF: https://www.ris.bka.gv.at/GeltendeFassung/LrSbg/20001389/%20

In [10]:
docs = [Document(page_content=text, metadata={"source_url": url})
        for url, text in all_sections]
print(f"Erstellt {len(docs)} LangChain-Dokumente.")

Erstellt 52 LangChain-Dokumente.


In [11]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=200)
chunked_docs = splitter.split_documents(docs)
print(f"Anzahl Chunks: {len(chunked_docs)}")

Anzahl Chunks: 156


In [12]:
for i, chunk in enumerate(chunked_docs):
    source = chunk.metadata["source_url"]
    text = chunk.page_content
    snippet = text[:500].replace("\n", " ")

    print(f"\n--- Chunk {i+1}/{len(chunked_docs)} ---")
    print(f"Quelle: {source}")
    print(f"Inhalt (erstes 500 Zeichen): {snippet} …")
    print(f"Länge dieses Chunks: {len(text)} Zeichen")
    print("--- Ende Chunk ---", flush=True)


--- Chunk 1/156 ---
Quelle: https://www.ris.bka.gv.at/GeltendeFassung/LrSbg/20001469/Zweitwohnung-Beschr%c3%a4nkungsgemeinden-Verordnung%202024%2c%20Fassung%20vom%2017.06.2025.pdf
Inhalt (erstes 500 Zeichen): Landesrecht Salzburg  www.ris.bka.gv.at Seite 1 von 1  Gesamte Rechtsvorschrift für Zweitwohnung -Beschränkungsgemeinden-Verordnung  2024, Fassung vom 17.06.2025  Langtitel  Verordnung der Salzburger Landessregierung vom 27. Februar 2024, mit der die Zweitwohnung- Beschränkungsgemeinden im Land Salzburg bestimmt werden (Zweitwohnung - Beschränkungsgemeinden-Verordnung 2024)  StF: LGBl Nr 26/2024  Präambel/Promulgationsklausel  Auf Grund des §  31 Abs  1 des Salzburger Raumordnungsgesetzes  2009  …
Länge dieses Chunks: 1135 Zeichen
--- Ende Chunk ---

--- Chunk 2/156 ---
Quelle: https://www.ris.bka.gv.at/GeltendeFassung/LrSbg/20001469/Zweitwohnung-Beschr%c3%a4nkungsgemeinden-Verordnung%202024%2c%20Fassung%20vom%2017.06.2025.pdf
Inhalt (erstes 500 Zeichen): 2. im Tennengau die Geme

In [13]:
class DebugOllamaEmbeddings(OllamaEmbeddings):
    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        total = len(texts)
        results = []
        for i, text in enumerate(texts, start=1):
            start_save_sg = time.perf_counter()
            print(f"Verarbeite Chunk {i}/{total}")
            emb = super().embed_documents([text])[0]
            results.append(emb)
            end_save_sg = time.perf_counter()
            print(f"Chunk {i} gespeichert in {end_save - start_save:.2f} Sekunden.")
        return results

In [14]:
embeddings = DebugOllamaEmbeddings(model=MODEL_NAME)

In [ ]:
start_embed = time.perf_counter()
print(f"Starte Embedding von {len(chunked_docs)} Chunks …")
# vector_store = FAISS.from_documents(chunked_docs, embeddings
vector_store = FAISS.from_documents(chunked_docs, embeddings, batch_size=32)
end_embed = time.perf_counter()
print(f"Embedding abgeschlossen in {end_embed - start_embed:.2f} Sekunden.")

start_save = time.perf_counter()
print("Speichere FAISS-Index …")
vector_store.save_local(INDEX_PATH)
end_save = time.perf_counter()
print(f"FAISS-Index gespeichert unter '{INDEX_PATH}' in {end_save - start_save:.2f} Sekunden.")
print(f"Gesamtzeit für Embedding + Speichern: {end_save - start_embed:.2f} Sekunden.")

Starte Embedding von 156 Chunks …
Verarbeite Chunk 1/156
Verarbeite Chunk 2/156
Verarbeite Chunk 3/156
Verarbeite Chunk 4/156
Verarbeite Chunk 5/156
Verarbeite Chunk 6/156
Verarbeite Chunk 7/156
Verarbeite Chunk 8/156
Verarbeite Chunk 9/156
Verarbeite Chunk 10/156
Verarbeite Chunk 11/156
Verarbeite Chunk 12/156
Verarbeite Chunk 13/156
Verarbeite Chunk 14/156
Verarbeite Chunk 15/156
Verarbeite Chunk 16/156
Verarbeite Chunk 17/156
Verarbeite Chunk 18/156
Verarbeite Chunk 19/156
Verarbeite Chunk 20/156
Verarbeite Chunk 21/156
Verarbeite Chunk 22/156
Verarbeite Chunk 23/156
Verarbeite Chunk 24/156
Verarbeite Chunk 25/156
Verarbeite Chunk 26/156
Verarbeite Chunk 27/156
Verarbeite Chunk 28/156
Verarbeite Chunk 29/156
Verarbeite Chunk 30/156
Verarbeite Chunk 31/156
Verarbeite Chunk 32/156
Verarbeite Chunk 33/156
Verarbeite Chunk 34/156
Verarbeite Chunk 35/156
Verarbeite Chunk 36/156
Verarbeite Chunk 37/156
Verarbeite Chunk 38/156
Verarbeite Chunk 39/156
Verarbeite Chunk 40/156
Verarbeite Chun

In [ ]:
query = "Welche Gemeinden im Pongau sind Zweitwohnung-Beschränkungsgemeinden?"

In [ ]:
results = vector_store.similarity_search_with_score(query, k=10)
print("\nTop-10 ähnliche Chunks für die Query:")
for rank, (doc, score) in enumerate(results, start=1):
    print(f"{rank}. Score={score:.2f}, Quelle={doc.metadata['source_url']}")
    print(">", doc.page_content[:120].replace("\n"," "), "…\n")

In [ ]:
retriever = vector_store.as_retriever(search_kwargs={"k":10})
qa_chain  = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever
)

In [ ]:
answer = qa_chain.run(query)
print("\nAntwort:", answer)